In [ ]:
!pip install -q -U google-generativeai

# Import the Python SDK
import os
import numpy as np
import pandas as pd
import google.generativeai as genai
import time
from requests.exceptions import ReadTimeout

userdata = {
    'SECRET_KEY': 'YOUR_SECRET_KEY'
}
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

# Base de conhecimento (exemplos)
documents = [
    {"Nome": "Rosa",
     "Rega": "Regar a cada 2-3 dias, mantendo o solo úmido, mas não encharcado.",
     "Sol": "Precisa de pelo menos 6 horas de sol direto por dia."},
    {"Nome": "Orquídea",
     "Rega": "Regar quando o substrato estiver quase seco, evitando encharcar as raízes.",
     "Sol": "Prefere luz indireta e brilhante, evitando sol direto intenso."},
    {"Nome": "Cacto",
     "Rega": "Regar somente quando o solo estiver completamente seco, evitando o excesso de água.",
     "Sol": "Adora sol pleno, tolerando até 6 horas de sol direto."},
    {"Nome": "Jiboia",
     "Rega": "Regar quando a camada superficial do solo estiver seca, evitando o encharcamento.",
     "Sol": "Prefere luz indireta, tolerando sombra parcial."},
    {"Nome": "Lavanda",
     "Rega": "Regar moderadamente, permitindo que o solo seque entre as regas.",
     "Sol": "Precisa de pelo menos 6-8 horas de sol direto por dia."},
    {"Nome": "Hortênsia",
     "Rega": "Regar frequentemente, mantendo o solo constantemente úmido.",
     "Sol": "Prefere sol pleno ou sombra parcial, especialmente em climas mais quentes."},
    {"Nome": "Manjericão",
     "Rega": "Regar regularmente, mantendo o solo úmido, mas não encharcado.",
     "Sol": "Precisa de pelo menos 6-8 horas de sol direto por dia."},
    {"Nome": "Samambaia",
     "Rega": "Manter o solo úmido, regando com frequência para evitar que seque.",
     "Sol": "Prefere luz indireta e sombra parcial, evitando sol direto."},
    {"Nome": "Alecrim",
     "Rega": "Regar moderadamente, permitindo que o solo seque entre as regas.",
     "Sol": "Precisa de pelo menos 6 horas de sol direto por dia."},
    {"Nome": "Espada-de-São-Jorge",
     "Rega": "Regar moderadamente, deixando o solo secar entre as regas.",
     "Sol": "Tolera desde luz indireta até sol direto, preferindo luz indireta brilhante."},
    {"Nome": "Hera",
     "Rega": "Manter o solo úmido, mas não encharcado, regando regularmente.",
     "Sol": "Prefere luz indireta brilhante, mas tolera sombra parcial."},
    {"Nome": "Gérbera",
     "Rega": "Regar regularmente, mantendo o solo úmido, mas evitando encharcar.",
     "Sol": "Prefere sol pleno, mas tolera sombra parcial em climas quentes."},
    {"Nome": "Cravo",
     "Rega": "Regar moderadamente, permitindo que o solo seque um pouco entre as regas.",
     "Sol": "Precisa de pelo menos 4-6 horas de sol direto por dia."},
    {"Nome": "Lírio-da-paz",
     "Rega": "Manter o solo úmido, regando quando a superfície estiver seca.",
     "Sol": "Prefere luz indireta, tolerando sombra parcial."},
    {"Nome": "Begônia",
     "Rega": "Manter o solo úmido, mas evitar o encharcamento.",
     "Sol": "Prefere luz indireta brilhante, evitando sol direto intenso."},
    {"Nome": "Crisântemo",
     "Rega": "Manter o solo uniformemente úmido, regando regularmente.",
     "Sol": "Prefere sol pleno, mas pode tolerar sombra parcial."},
    {"Nome": "Peperômia",
     "Rega": "Regar quando o solo estiver seco ao toque, evitando excesso de água.",
     "Sol": "Prefere luz indireta brilhante, mas pode tolerar sombra parcial."},
    {"Nome": "Zamioculca",
     "Rega": "Regar moderadamente, permitindo que o solo seque entre as regas.",
     "Sol": "Tolera baixa luminosidade, mas prefere luz indireta."},
    {"Nome": "Gerânio",
     "Rega": "Regar quando o solo estiver seco ao toque, evitando encharcar.",
     "Sol": "Precisa de pelo menos 6 horas de sol direto por dia."},
    {"Nome": "Ficus",
     "Rega": "Manter o solo levemente úmido, regando quando a superfície estiver seca.",
     "Sol": "Prefere luz indireta brilhante, mas pode tolerar sombra parcial."},
    {"Nome": "Suculentas",
     "Rega": "Regar apenas quando o solo estiver completamente seco.",
     "Sol": "Precisa de pelo menos 4-6 horas de sol direto por dia."},
    {"Nome": "Costela-de-Adão",
     "Rega": "Manter o solo úmido, regando quando a superfície estiver seca.",
     "Sol": "Prefere luz indireta brilhante, mas tolera sombra parcial."},
    {"Nome": "Calanchoê",
     "Rega": "Regar moderadamente, permitindo que o solo seque entre as regas.",
     "Sol": "Prefere sol pleno ou luz indireta brilhante."},
    {"Nome": "Bromélia",
     "Rega": "Manter o solo levemente úmido, mas evitar o encharcamento.",
     "Sol": "Prefere luz indireta brilhante, mas pode tolerar sol direto suave."},
    {"Nome": "Violeta-africana",
     "Rega": "Manter o solo levemente úmido, evitando molhar as folhas.",
     "Sol": "Prefere luz indireta brilhante, evitando sol direto intenso."},
    {"Nome": "Maranta",
     "Rega": "Manter o solo constantemente úmido, evitando encharcar.",
     "Sol": "Prefere luz indireta e sombra parcial."},
    {"Nome": "Palmeira-ráfis",
     "Rega": "Manter o solo úmido, regando quando a superfície estiver seca.",
     "Sol": "Prefere luz indireta, mas tolera sombra parcial."},
    {"Nome": "Fitônia",
     "Rega": "Manter o solo constantemente úmido, evitando encharcar.",
     "Sol": "Prefere luz indireta brilhante, mas pode tolerar sombra parcial."},
    {"Nome": "Petúnia",
     "Rega": "Regar regularmente, mantendo o solo úmido, mas evitando encharcar.",
     "Sol": "Prefere sol pleno, mas pode tolerar sombra parcial."},
    {"Nome": "Espatifilo",
     "Rega": "Manter o solo úmido, regando quando a superfície estiver seca.",
     "Sol": "Prefere luz indireta brilhante, mas pode tolerar sombra."}
]

# Converter os nomes das plantas para minúsculas
for document in documents:
    document["Nome"] = document["Nome"].lower()

# Criar DataFrame
df = pd.DataFrame(documents)

# Gerar embeddings para cada documento
model = "models/embedding-001"

def embed_fn(nome, text, retries=3):
    for attempt in range(retries):
        try:
            response = genai.embed_content(model=model, content=text, title=nome, task_type="RETRIEVAL_DOCUMENT")
            if "embedding" in response:
                return response["embedding"]
            else:
                raise ValueError("Resposta da API não contém 'embedding'")
        except ReadTimeout as e:
            print(f"Timeout occurred on attempt {attempt + 1}/{retries}: {e}")
            if attempt < retries - 1:
                time.sleep(2)  # Aguarde um pouco antes de tentar novamente
            else:
                raise

df["Embeddings"] = df.apply(lambda row: embed_fn(row["Nome"], f"Rega: {row['Rega']}\nSol: {row['Sol']}"), axis=1)

# Função para encontrar a melhor resposta
def gerar_e_buscar_consulta(consulta, base, model):
    consulta = consulta.lower()
    # Verificar se o nome da planta está no DataFrame
    if consulta not in base["Nome"].values:
        return "Essa planta não existe na minha memória :)"

    try:
        response = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")
        if "embedding" not in response:
            raise ValueError("Resposta da API não contém 'embedding'")
        embedding_da_consulta = response["embedding"]
    except Exception as e:
        raise ValueError(f"Erro ao gerar embedding da consulta: {e}")



    # Ajuste para acessar as colunas corretamente
    produtos_escalares = np.dot(np.stack(base["Embeddings"].values), embedding_da_consulta)

    indice = np.argmax(produtos_escalares)
    return base.iloc[indice]

# Interface de chat (simplificada)
while True:
    consulta = input("Digite o nome da planta: ")
    if consulta.lower() == "sair":
        break

    # Encontrar a melhor resposta
    try:
        resposta = gerar_e_buscar_consulta(consulta, df, model)
        print(f"\n--- {resposta['Nome'].upper()} ---")
        print(f"**Rega:** {resposta['Rega']}")
        print(f"**Sol:** {resposta['Sol']}\n")
    except Exception as e:
        print(f"Erro ao buscar a planta: {e}")
        # Se não encontrar uma resposta exata, usar Gemini para gerar uma
        generation_config = {
            "temperature": 0,
            "candidate_count": 1
        }
        try:
            model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
            resposta = model_2.generate_content(f"Responda com as informações de rega e exposição ao sol para a planta '{consulta}'")

            # Verificação e impressão do texto da resposta
            if hasattr(resposta, 'text'):
                print(f"\n--- {consulta.upper()} ---")
                print(f"{resposta.text}\n")
            else:
                print(f"Erro: A resposta gerada não contém texto.\n")
        except Exception as e:
            print(f"Erro ao gerar a resposta usando o modelo Gemini: {e}")

Digite o nome da planta: Rosa

--- ROSA ---
**Rega:** Regar a cada 2-3 dias, mantendo o solo úmido, mas não encharcado.
**Sol:** Precisa de pelo menos 6 horas de sol direto por dia.

Digite o nome da planta: rosa

--- ROSA ---
**Rega:** Regar a cada 2-3 dias, mantendo o solo úmido, mas não encharcado.
**Sol:** Precisa de pelo menos 6 horas de sol direto por dia.

Digite o nome da planta: ROSA

--- ROSA ---
**Rega:** Regar a cada 2-3 dias, mantendo o solo úmido, mas não encharcado.
**Sol:** Precisa de pelo menos 6 horas de sol direto por dia.

